In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os

# import functions
from pysrc.load_data import load_data
from pysrc.load_spec import load_spec
# from pysrc.dfm import dfm
from pysrc.summarize import summarize
from pysrc.dfm import dfm

import plotly.graph_objects as go

np.seterr(over='raise', under='raise', invalid='raise')

{'divide': 'warn', 'over': 'raise', 'under': 'raise', 'invalid': 'raise'}

In [8]:
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.subplots as psub


In [11]:
def plot(
    dt: pd.Series,
    common_factor: pd.Series,
    series: pd.Series,
    series_name: str = "",
    yaxis_title: str="",
    mode="markers",
    xtickfont_size=14,
    ytickfont_size=14,
) -> None:
    body = [
        go.Scatter(
            name="Common Factor",
            x=dt,
            y=common_factor,
            mode=mode,
            # line=dict(color='rgb(225, 69, 0)', width=2),  # rgb(216, 129, 71)
            line=dict(color="#841E62", width=1.25),
            marker={"size": 5},
            opacity=0.85,
        ),
    ]

    body.append(
        go.Scatter(
            name=series_name,
            x=dt,
            y=series,
            mode=mode,
            marker={"size": 5, "symbol": "diamond"},
            line=dict(color="#000000", width=1.25),  # #7BCC62 / #68b562 / #7BB562
            opacity=0.85,
        )
    )

    fig = go.Figure(body)

    fig.update_layout(
        autosize=False,
        width=900,
        height=650,
        plot_bgcolor="white",
        yaxis_title=yaxis_title,
        title=f"Projection of common factor onto {series_name}",
        hovermode="x",
    )
    fig.update_xaxes(
        mirror=True,
        ticks="outside",
        showline=True,
        linecolor="black",
        gridcolor="lightgrey",
        tickfont_size=xtickfont_size,
    )
    fig.update_yaxes(
        mirror=True,
        ticks="outside",
        showline=True,
        linecolor="black",
        gridcolor="lightgrey",
        tickfont_size=ytickfont_size,
    )

    fig.show()
    # return fig

In [13]:
### Dynamic factor model (DFM) ############################################
# This script estimates a dynamic factor model (DFM) using a panel of
# monthly and quarterly series.
###########################################################################


## Clear workspace and set paths.
# close all; clear; clc;
# addpath('functions');


## User inputs.
vintage = '2016-06-29'; # vintage dataset to use for estimation
country = 'US';         # United States macroeconomic data
sample_start = datetime.strptime('2000-01-01', '%Y-%m-%d'); # estimation sample


## Load model specification and dataset.
# Load model specification structure `Spec`
Spec = load_spec('Spec_US_example.xls');
# Parse `Spec`
SeriesID, SeriesName, Units, UnitsTransformed = Spec['seriesid'], Spec['seriesname'], Spec['units'], Spec['unitstransformed']

# Load data
datafile = os.path.join('data', country, f'{vintage}.xls');
X, Time, Z = load_data(datafile, Spec, sample_start);
summarize(X,Time,Spec,vintage); # summarize data


## Plot raw and transformed data.
# Industrial Production (INDPRO) <fred.stlouisfed.org/series/INDPRO>
seriesName = "INDPRO"
idxSeries = SeriesID.index(seriesName)
t_obs = ~np.isnan(X[:, idxSeries])

fig = psub.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                            subplot_titles=("Raw Observed Data", "Transformed Data"))

# Plot raw observed data
trace1 = go.Scatter(
    x=Time[t_obs],
    y=Z[t_obs, idxSeries],
    mode='lines',
    name='Raw Observed Data',
    line=dict(color="#000000", width=1),  # #7BCC62 / #68b562 / #7BB562
)
fig.add_trace(trace1, row=1, col=1)

# Plot transformed data
trace2 = go.Scatter(
    x=Time[t_obs],
    y=X[t_obs, idxSeries],
    mode='lines',
    name='Transformed Data',
    line=dict(color="#BDC1D6", width=1),  # #7BCC62 / #68b562 / #7BB562
)
fig.add_trace(trace2, row=2, col=1)
fig.update_layout(
    height=600,
    width=800,
    showlegend=False,
    title_text=seriesName,
    plot_bgcolor="white",
)
fig.update_xaxes(range=[Time[0], Time[-1]], title_text='Time', row=2, col=1, gridcolor="lightgrey")
fig.update_xaxes(range=[Time[0], Time[-1]], title_text='Time', row=1, col=1, gridcolor="lightgrey")
fig.update_yaxes(title_text=Units[idxSeries], row=1, col=1, gridcolor="lightgrey")
fig.update_yaxes(title_text=UnitsTransformed[idxSeries], row=2, col=1, gridcolor="lightgrey")
fig.show()

## Run dynamic factor model (DFM) and save estimation output as 'ResDFM'.
# threshold = 1e-4; # Set to 1e-5 for more robust estimates
threshold = 1e-5;

Res = dfm(X,Spec, max_iter=0);
# save('ResDFM','Res','Spec');

## Plot common factor and standardized data.
traces = []
for i in range(Res['x_sm'].shape[1]):
    trace = go.Scatter(
        x=Time,
        y=Res['x_sm'][:, i],
        mode='lines',
        line=dict(width=0.75),
        name=f'Standardized Data {i + 1}'  # Label each line
    )
    traces.append(trace)

# Add traces for each row of the common factor data
common_factor_data = Res['Z'][:, :] * Res['C'][idxSeries, :]  # Matrix multiplication
for i in range(common_factor_data.shape[1]):
    trace_common_factor = go.Scatter(
        x=Time,
        y=common_factor_data[:, i],
        mode='lines',
        line=dict(color='black', width=1),
        name=f'Common Factor {i + 1}'
    )
    traces.append(trace_common_factor)

# Define the layout of the plot
layout = go.Layout(
    title='Common Factor and Standardized Data',
    xaxis=dict(title='Time', range=[Time[0], Time[-1]]),
    yaxis=dict(title='Values'),
    showlegend=True,
    boxmode='group',
    plot_bgcolor="white",
)

# Create the figure with the defined traces and layout
fig = go.Figure(data=traces, layout=layout)
fig.update_xaxes(
    mirror=True,
    ticks="outside",
    showline=True,
    linecolor="black",
    gridcolor="lightgrey",
)
fig.update_yaxes(
    mirror=True,
    ticks="outside",
    showline=True,
    linecolor="black",
    gridcolor="lightgrey",
)

# Display the plot
pyo.iplot(fig)

## Common Factor Projection: plot projection of common factor onto Payroll Employment and GDP.

## First subplot - projection of common factor onto PAYEMS
# nameSeries = "PAYEMS"
# idxSeries = SeriesID.index(nameSeries)
# t_obs = ~np.isnan(X[:, idxSeries])
# CommonFactor = Res["C"][idxSeries, :5] @ Res["Z"][:, :5].T * Res["Wx"][idxSeries] + Res["Mx"][idxSeries]
# plot(dt=Time, series=X[:, idxSeries], common_factor=CommonFactor, mode = "lines", series_name=nameSeries, yaxis_title=f'{Units[idxSeries]}, {UnitsTransformed[idxSeries]}')

## Second subplot - projection of common factor onto GDPC1
nameSeries = "GDPC1"
idxSeries = SeriesID.index(nameSeries)
t_obs = ~np.isnan(X[:, idxSeries])
CommonFactor = Res["C"][idxSeries, :5] @ Res["Z"][:, :5].T * Res["Wx"][idxSeries] + Res["Mx"][idxSeries]
plot(dt=Time, series=X[:, idxSeries], common_factor=CommonFactor, mode = "lines+markers", series_name=nameSeries, yaxis_title=f'{Units[idxSeries]}, {UnitsTransformed[idxSeries]}')


Table 1: Model specification
              SeriesID                   SeriesName                 Units  \
0               PAYEMS           Payroll Employment  Thousands of Persons   
1               JTSJOL                 Job Openings             Thousands   
2             CPIAUCSL         Consumer Price Index                 Index   
3              DGORDER         Durable Goods Orders           $, Millions   
4                RSAFS                 Retail Sales           $, Millions   
5               UNRATE            Unemployment Rate                     %   
6                HOUST               Housing Starts    Thousands of Units   
7               INDPRO        Industrial Production                 Index   
8              DSPIC96              Personal Income   Chained $, Billions   
9              BOPTEXP                      Exports           $, Millions   
10             BOPTIMP                      Imports           $, Millions   
11             TTLCONS        Construction Spen

/Users/ejowik001/Desktop/Github/Nowcasting/pysrc/load_spec.py:41: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`






Table 3: Block Loading Structure
                             1global  2soft  3real  4labor
Payroll_Employment                 1      0      0       1
Job_Openings                       1      0      0       1
Consumer_Price_Index               1      0      0       0
Durable_Goods_Orders               1      0      1       0
Retail_Sales                       1      0      1       0
Unemployment_Rate                  1      0      0       1
Housing_Starts                     1      0      1       0
Industrial_Production              1      0      1       0
Personal_Income                    1      0      1       0
Exports                            1      0      1       0
Imports                            1      0      1       0
Construction_Spending              1      0      1       0
Import_Price_Index                 1      0      0       0
Core_Consumer_Price_Index          1      0      0       0
Core_PCE_Price_Index               1      0      0       0
PCE_Price_Index     